In [97]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

# Iterative Deepening

In [98]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random
# Pkg.add("DataStructures")
using DataStructures
# Pkg.add("NBInclude")
using NBInclude

In [99]:
@nbinclude("EvaluatePosition.ipynb")

terminal_evaluation (generic function with 1 method)

In [194]:
@nbinclude("AlphaBetaPruning.ipynb")

alphaBetaPruning (generic function with 1 method)

## maxValue

The `maxValue` function takes in 3 arguments and 2 optional arguments.
1. `State` is a chess `state` of type `Board`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating 
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process
        
The function returns the maximal centipawn evaluation of the current position for the player playing white where both players have played the optimal moves according to the algorithm and terminating after the given depth. All possible moves are sorted by a `PriorityQueue` using the evaluation of the position. Good moves will be prioritized which will increase the chance of pruning paths. 

In [119]:
function maxValue(State, score, depth, alpha=-Inf, beta=Inf)
    if isterminal(State) 
        return terminal_evaluation(State) 
    end
    if depth == 0
        return score
    end
    value = alpha
    queue = PriorityQueue()
    for move in moves(State)
        undoinfo = domove!(State, move)
        val = value_cache(State, depth-1)
        undomove!(State, undoinfo)
        if val == nothing
            val = -Inf
        end
        enqueue!(queue, move, -val)
    end
    while !isempty(queue)
        move = peek(queue)[1]
        nextEval = evaluate_move(State, move, score)
        undoinfo = domove!(State, move)
        value = max(value, evaluate(State, minValue, nextEval, depth - 1, value, beta))
        undomove!(State, undoinfo)
        if value >= beta
            return value
        end
        delete!(queue, move)
    end
    return value
end

maxValue (generic function with 3 methods)

## minValue

The `minValue` function takes in 3 arguments and 2 optional arguments.
1. `State` is a chess `state` of type `Board`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating 
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process
        
The function returns the maximal centipawn evaluation of the current position for the player playing white where both players have played the optimal moves according to the algorithm and terminating after the given depth. All possible moves are sorted by a `PriorityQueue` using the evaluation of the position. Good moves will be prioritized which will increase the chance of pruning paths. 

In [120]:
function minValue(State, score, depth, alpha=-Inf, beta=Inf)
    if isterminal(State)
        return terminal_evaluation(State)
    end
    if depth == 0
        return score
    end
    value = beta
    queue = PriorityQueue()
    for move in moves(State)
        undoinfo = domove!(State, move)
        val = value_cache(State, depth-1)
        undomove!(State, undoinfo)
        if val == nothing
            val = Inf
        end
        enqueue!(queue, move, val)
    end
    while !isempty(queue)
        move = peek(queue)[1]
        nextEval = evaluate_move(State, move, score)
        undoinfo = domove!(State, move)
        value = min(value, evaluate(State, maxValue, nextEval, depth - 1, alpha, value))
        undomove!(State, undoinfo)
        if value <= alpha
            return value
        end
        delete!(queue, move)
    end
    return value
end

minValue (generic function with 3 methods)

The function `value_cache` is a helping function for the `minValue` and `maxValue` function. It takes in 2 arguments:
1. `State` is a chess `state` of type `Board`
1. `depth` is the number of halfmoves the engine should analyze before terminating

The function looks into the Cache and returns any previously saved values for this position. This information is used to sort good moves inside of the PriorityQueue.
The function returns the value for this `state` if the `state` is in the Cache and has a sufficient pre-calculated depth. If the cache does not have an entry for this `state` or entry does not have sufficient depth the function will return `nothing`.

In [121]:
function value_cache(State, depth)
    global gCache
    hash = zobrist_hash(State, zobristHasher)
    if hash in keys(gCache)
        _, value, d = gCache[hash]
        if d >= depth
            return value
        end
    end
    # new move or no entry with enough depth
    return nothing
end

value_cache (generic function with 1 method)

The `alphaBetaPruning` function takes in 3 arguments
1. `State` is the current state of type `Board`
1. `score` is the static centipawn evaluation of the static position
1. `depth` is the number of halfmoves the engine should analyze before terminating

The function returns the best value and the best move the moving player can play in the current position. It calls the `Iterative Deepening` algorithm. If multiple moves are found which result in the best evaluation a random move will be chosen.

In [189]:
function pd_evaluate(State, f,score ,depth)
    bestVal = score
    for d in 0:depth
        bestVal = evaluate(State,f, score, d)
        if bestVal in [100000, -100000]
            return bestVal
        end
    end
    return bestVal
end

pd_evaluate (generic function with 2 methods)

In [191]:
function iterativeDeepening(State, score, depth)
    side = sidetomove(State)
    bestVal = (side == WHITE) ? pd_evaluate(State, maxValue, score, depth) : pd_evaluate(State, minValue, score, depth)
    next_moves = moves(State)
    BestMoves = []
    for move in next_moves
        nextEval = evaluate_move(State, move, score)
        undoinfo = domove!(State, move)
        if (side == WHITE && evaluate(State, minValue, nextEval, depth - 1) == bestVal) ||
            (side == BLACK && evaluate(State, maxValue, nextEval, depth - 1) == bestVal)
            append!(BestMoves, [move])
        end
        undomove!(State, undoinfo)
    end
    println("BestMoves: ", BestMoves)
    BestMove = rand(BestMoves)
    return bestVal, BestMove
end

iterativeDeepening (generic function with 1 method)

In [197]:
gCache = Dict()
b2 = fromfen("k7/7K/8/8/6r1/8/8/5r2 b - - 0 1")
display(b2)
iterativeDeepening(b2, evaluate_position(b2), 4)

Board (k7/7K/8/8/6r1/8/8/5r2 b - -):
 k  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  K 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  r  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  r  -  -

BestMoves: Any[Move(f1a1), Move(f1b1), Move(f1c1), Move(f1d1), Move(f1e1), Move(f1f8), Move(f1f6), Move(f1f3), Move(f1f2), Move(f1g1), Move(f1h1), Move(g4g3), Move(g4g2), Move(a8a7), Move(a8b8), Move(a8b7)]


(-100000.0, Move(f1f3))